In [1]:
import os
import glob
import rasterio as rio
from rasterio.mask import raster_geometry_mask
from rasterio import plot
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import pyproj
from shapely.geometry import shape, MultiPolygon
from shapely.ops import transform

In [6]:
# Functions

# Get panel masks

def get_panel_masks(geojson_path, ref_img_path):
    '''This function generates a mask that leaves only pixels inside 
    vine panels unmasked. 
    
    Inputs:
    geojson_path: path to a geojson file with the panel geometries. 
    Must be in same CRS as image, otherwise un-comment code section starting with "dst_crs".
    ref_img_path: path to the .tif file for the image
    
    Outputs:
    A set of masks, as Boolean arrays. True values are masked pixels.
    False values are unmasked pixels.
    '''
    with open(geojson_path, 'r') as f:
        panels = json.load(f)
        features = panels['features']
        features_ordered = sorted(features, key= lambda feature:(feature['properties']['row'],feature['properties']['panel'])) # ground truth order.
        panel_aois = [f['geometry'] for f in features_ordered]
    
    with rio.open(ref_img_path) as src:
        dst_crs = src.crs
        
        # geojson features (the field block geometries)
        # are often in WGS84
        # project these to the image coordinates
        wgs84 = pyproj.CRS('EPSG:4326')

        project = pyproj.Transformer.from_crs(wgs84, dst_crs, always_xy=True).transform
        proj_panel_aois = [transform(project, shape(p)) for p in panel_aois]

        masks = [raster_geometry_mask(src, [p], crop=False)[0]
                 for p in proj_panel_aois]
       
    return masks
   

# Get reflectance df

def get_rfl(img_path, masks):
    '''
    Function to extract per-panel reflectance values for 4-band SkySat images.
    Calculates the mean reflectance values for the pixels in each panel.
    The output is formatted as a pandas dataframe.
    
    Inputs:
    img_path (str) - abs. path to the image       
    masks (list) - a list of masks that correspond to the panel aois. Each mask item is a Boolean array. 
    '''
    
    with rio.open(img_path,'r') as img:
        def _get_stats(mask, panel_number):
            panel_blue = np.ma.mean(np.ma.array(img.read(1), mask=mask))#/10000
            panel_green = np.ma.mean(np.ma.array(img.read(2), mask=mask))#/10000
            panel_red = np.ma.mean(np.ma.array(img.read(3), mask=mask))#/10000
            panel_nir = np.ma.mean(np.ma.array(img.read(4), mask=mask))#/10000
            
            return {'acquired': img_path.split('/')[-1].split('_')[-5], # date from image filename
                    'plot': panel_number + 1,    #add one - panels are numbered starting at 1 in the 'real world'
                    'blue': panel_blue,
                    'green': panel_green,
                    'red': panel_red,
                    'nir': panel_nir}
        data = [_get_stats(m, i) for i, m in enumerate(masks)]  # sets the parameters for the nested function as the mask (m) and corresponding panel number (i). 
        df = pd.DataFrame(data)
    
    return df

In [7]:
# image directory containing co-registered SkySat scenes 

img_dir = '../data/images/2020/SkySat/'

# image pathnames
img_paths = glob.glob(img_dir+'NORM*.tif')

# geojson with panel geometries

#json_path = '/Users/kathleenkanaley/Desktop/grapes_in_space/map/qgis_layers/crit_panels_geojson.geojson'
json_path = '../data/crit_panels_geojson.geojson'

img_paths


['../data/images/2020/SkySat/NORM_minmax_20200710_155028_ssc3d2_0012_ar.tif',
 '../data/images/2020/SkySat/NORM_minmax_20200625_154704_ssc12d2_0013_ar.tif',
 '../data/images/2020/SkySat/NORM_minmax_20200812_153924_ssc3d2_0013_ar.tif']

In [4]:
json_path

'../data/crit_panels_geojson.geojson'

In [8]:
# Get dataframes for all images in directory

master_df = pd.DataFrame()

for i in img_paths:
    masks = get_panel_masks(json_path, i)
    df = get_rfl(i, masks)
    master_df = pd.concat([master_df, df], axis=0)
    master_df = master_df.sort_values(by=['acquired', 'plot'])

master_df

,acquired,plot,blue,green,red,nir
0,20200625,1,0.398813,0.519112,0.416702,0.658977
1,20200625,2,0.432868,0.533166,0.433583,0.616750
2,20200625,3,0.402119,0.506342,0.399205,0.602489
3,20200625,4,0.403295,0.526952,0.417283,0.620601
4,20200625,5,0.365009,0.483301,0.337710,0.677268
...,...,...,...,...,...,...
315,20200812,316,0.232366,0.387582,0.257785,0.705615
316,20200812,317,0.231307,0.402387,0.256544,0.739210
317,20200812,318,0.232197,0.394040,0.236985,0.736111
318,20200812,319,0.239480,0.403399,0.245549,0.771425


In [9]:
# Append corresponding row and panel numbers for each plot 

panel_csv = pd.read_csv('../data/crit_panels.csv')
cols = panel_csv.iloc[:,1:] # adjust this according to the .csv
#cols
appended = pd.concat([master_df, cols],axis=1)
appended

,acquired,plot,blue,green,red,nir,row,panel
0,20200625,1,0.398813,0.519112,0.416702,0.658977,1,1
1,20200625,2,0.432868,0.533166,0.433583,0.616750,1,2
2,20200625,3,0.402119,0.506342,0.399205,0.602489,1,3
3,20200625,4,0.403295,0.526952,0.417283,0.620601,1,4
4,20200625,5,0.365009,0.483301,0.337710,0.677268,1,5
...,...,...,...,...,...,...,...,...
315,20200812,316,0.232366,0.387582,0.257785,0.705615,20,12
316,20200812,317,0.231307,0.402387,0.256544,0.739210,20,13
317,20200812,318,0.232197,0.394040,0.236985,0.736111,20,14
318,20200812,319,0.239480,0.403399,0.245549,0.771425,20,15


In [10]:
appended.acquired.unique()


array(['20200625', '20200710', '20200812'], dtype=object)

In [11]:
scout_csv = pd.read_csv('../data/scout/scout_2020_gdf.csv')
print(scout_csv.Date.unique())
print(scout_csv.columns)

['2020-06-18' '2020-06-25' '2020-07-01' '2020-07-09' '2020-07-15'
 '2020-07-21' '2020-07-30' '2020-08-06' '2020-08-13' '2020-08-20'
 '2020-09-09']
Index(['Date', 'Row', 'Panel', 'Treatment', 'Block', 'PM_severity',
       'DM_severity', 'total_dis', 'geometry', 'centroid'],
      dtype='object')


In [12]:
# Match scout data to image data

# Define filtering and matching parameters

first_row = 1
last_row = 10

match_dates = ['2020-06-25','2020-07-09', '2020-08-13']
match_cols =['row', 'panel', 'Date']


# Filter and join datasets

img_filtered = appended[appended['row']>=first_row]
img_filtered = img_filtered[img_filtered['row']<=last_row]
scout_filtered = scout_csv[scout_csv['Date'].isin(match_dates)]
scout_filtered = scout_filtered.rename(columns={'Row':'row', 'Panel':'panel'})

# Create a 'Date' column in the image dataframe to match the scout date
def match_days(row):
    if row['acquired']=='20200625':
        return '2020-06-25'
    if row['acquired']=='20200710':
        return '2020-07-09'
    if row['acquired']=='20200812':
        return '2020-08-13'

img_filtered['Date'] = img_filtered.apply(lambda row: match_days(row), axis=1)

# Join and drop NANs + unneeded columns
joined = img_filtered.join(scout_filtered.set_index(match_cols), on=match_cols)
cleaned = joined.dropna(axis=0,subset=['Treatment', 'Block', 'DM_severity', 'PM_severity', 'total_dis', 'geometry', 'centroid'])
#cleaned = joined.dropna(axis=0, subset=['geometry','Block', 'total_dis','centroid'])

In [13]:
cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 299 entries, 1 to 158
Data columns (total 16 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   acquired     299 non-null    object 
 1   plot         299 non-null    int64  
 2   blue         299 non-null    float64
 3   green        299 non-null    float64
 4   red          299 non-null    float64
 5   nir          299 non-null    float64
 6   row          299 non-null    int64  
 7   panel        299 non-null    int64  
 8   Date         299 non-null    object 
 9   Treatment    299 non-null    float64
 10  Block        299 non-null    object 
 11  PM_severity  299 non-null    float64
 12  DM_severity  299 non-null    float64
 13  total_dis    299 non-null    float64
 14  geometry     299 non-null    object 
 15  centroid     299 non-null    object 
dtypes: float64(8), int64(3), object(5)
memory usage: 39.7+ KB


In [14]:
# If using a raster with masked pixels

no_nullpix = cleaned.dropna(axis=0, subset=['blue', 'green', 'red', 'nir'])
no_nullpix.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 299 entries, 1 to 158
Data columns (total 16 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   acquired     299 non-null    object 
 1   plot         299 non-null    int64  
 2   blue         299 non-null    float64
 3   green        299 non-null    float64
 4   red          299 non-null    float64
 5   nir          299 non-null    float64
 6   row          299 non-null    int64  
 7   panel        299 non-null    int64  
 8   Date         299 non-null    object 
 9   Treatment    299 non-null    float64
 10  Block        299 non-null    object 
 11  PM_severity  299 non-null    float64
 12  DM_severity  299 non-null    float64
 13  total_dis    299 non-null    float64
 14  geometry     299 non-null    object 
 15  centroid     299 non-null    object 
dtypes: float64(8), int64(3), object(5)
memory usage: 39.7+ KB


In [15]:
# Export dataframe as .csv

output_fp = '../data/img_scout_dfs/2020/'

cleaned.to_csv(output_fp+'NORM_skysat_scout_2020.csv', index=False)